In [1]:
pip install --upgrade pythainlp

In [2]:
pip install pyLDAvis


In [3]:
import pandas as pd
import pythainlp


In [4]:
import gensim

In [5]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

In [6]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [7]:
df = pd.read_csv('review.csv')

In [8]:
print(df.tail(10))

   Review ID                                             Review
0          1  โรงแรมสวยสะอาดดี ติดทะเล แต่เล่นน้ำทะเลไม่ได้ต...
1          2  โรงแรมคือดีเลย ติดริมทะเล แต่ติดตรงที่ชายหาดส่...
2          3  สิ่งที่ชอบที่สุดของโรงแรมนี้คือพนักงาน บริการด...
3          4  บริการดี ตั้งแต่ก้าวแรกที่เดินลงไปที่โรงแรมเลย...
4          5  เข้าพักที่ Cape dara resort มิถุนายน 2021 ประท...
5          6  มาพักครั้งแรก ประทับใจการบริการและการต้อนรับ ท...
6          7  ป็นโรงแรมที่คิดมาตลอดว่า ต้องมาพักดูสักคืน โดย...
7          8  เข้าพักทั้งครอบครัวเราพักที่นี่ประจำ มาวันธรรม...
8          9  ห้องสวยมาก แต่หาดเฉยๆ เหมาะกับการมาเปลี่ยนที่น...
9         10  ประทับใจบริการทุกครั้งที่ได้ไปพัก ทุกแผนกเลย ห...


In [9]:
stopwords = list(pythainlp.corpus.thai_stopwords())
removed_words = ['',' ', '\n', 'โรงเเรม', '(', ')']
screening_words = stopwords + removed_words

def tokenize_with_space(sentence):
  merged = ''
  words = pythainlp.word_tokenize(str(sentence), engine='newmm')
  for word in words:
    if word not in screening_words:
      merged = merged + ','+ word
      return merged[1:]


In [10]:
df['Review_tokenized'] = df['Review'].apply(lambda x: tokenize_with_space(x))


In [11]:
df.tail()

,Review ID,Review,Review_tokenized
5,6,มาพักครั้งแรก ประทับใจการบริการและการต้อนรับ ท...,พัก
6,7,ป็นโรงแรมที่คิดมาตลอดว่า ต้องมาพักดูสักคืน โดย...,ป็น
7,8,เข้าพักทั้งครอบครัวเราพักที่นี่ประจำ มาวันธรรม...,พัก
8,9,ห้องสวยมาก แต่หาดเฉยๆ เหมาะกับการมาเปลี่ยนที่น...,ห้อง
9,10,ประทับใจบริการทุกครั้งที่ได้ไปพัก ทุกแผนกเลย ห...,ประทับใจ


In [12]:
documents = df['Review_tokenized'].to_list()
texts = [[text for text in doc.split(',')] for doc in documents]
dictionary = gensim.corpora.Dictionary(texts)

In [13]:
print(dictionary.token2id.keys())

dict_keys(['โรงแรม', 'ชอบ', 'บริการ', 'พัก', 'ป็น', 'ห้อง', 'ประทับใจ'])


In [14]:
gensim_corpus = [dictionary.doc2bow(text,allow_update=True) for text in texts]
word_frequencies = [[(dictionary[id], frequence) for id, frequence in couple] for couple in gensim_corpus]

In [15]:
num_topics = 10
chunksize = 2000
passes = 10
iterations = 50
eval_every = 1

In [16]:
temp = dictionary[0]
id2word = dictionary.id2token

model = gensim.models.LdaModel(
    corpus=gensim_corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    passes=passes,
    eval_every=eval_every
)


In [17]:
pyLDAvis.gensim.prepare(model, gensim_corpus, dictionary)

PreparedData(topic_coordinates=         x    y  topics  cluster  Freq
topic                                 
0      0.0  0.0       1        1   1.0
63     0.0  0.0       2        1   1.0
73     0.0  0.0       3        1   1.0
72     0.0  0.0       4        1   1.0
71     0.0  0.0       5        1   1.0
...    ...  ...     ...      ...   ...
30     0.0  0.0      96        1   1.0
29     0.0  0.0      97        1   1.0
28     0.0  0.0      98        1   1.0
27     0.0  0.0      99        1   1.0
99     0.0  0.0     100        1   1.0

[100 rows x 5 columns], topic_info=        Term      Freq     Total  Category  logprob  loglift
0     โรงแรม  1.000000  1.000000   Default   7.0000      7.0
1        ชอบ  1.000000  1.000000   Default   6.0000      6.0
2     บริการ  1.000000  1.000000   Default   5.0000      5.0
3        พัก  1.000000  1.000000   Default   4.0000      4.0
4        ป็น  1.000000  1.000000   Default   3.0000      3.0
..       ...       ...       ...       ...      ...      ...
2     บริการ  0.014286  1.428572  Topic100  -1.9459      0.0
3        พัก  0.014286  1.428572  Topic100  -1.9459      0.0
4        ป็น  0.014286  1.428572  Topic100  -1.9459      0.0
5       ห้อง  0.014286  1.428572  Topic100  -1.9459      0.0
6   ประทับใจ  0.014286  1.428572  Topic100  -1.9459      0.0

[707 rows x 6 columns], token_table=Empty DataFrame
Columns: [Topic, Freq, Term]
Index: [], R=7, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 64, 74, 73, 72, 71, 70, 69, 68, 67, 66, 65, 63, 2, 62, 61, 60, 59, 58, 57, 56, 55, 54, 53, 75, 76, 77, 78, 99, 98, 97, 96, 95, 94, 93, 92, 91, 90, 89, 88, 87, 86, 85, 84, 83, 82, 81, 80, 79, 52, 51, 50, 25, 23, 22, 21, 20, 19, 18, 17, 16, 15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 24, 26, 49, 27, 48, 47, 46, 45, 44, 43, 42, 41, 40, 39, 38, 37, 36, 35, 34, 33, 32, 31, 30, 29, 28, 100])

In [18]:
model.show_topic(1)

[('โรงแรม', 0.14285715),
 ('ชอบ', 0.14285715),
 ('บริการ', 0.14285715),
 ('พัก', 0.14285715),
 ('ป็น', 0.14285715),
 ('ห้อง', 0.14285715),
 ('ประทับใจ', 0.14285715)]

In [22]:
def get_topic_and_score(document):
    doc_bow = dictionary.doc2bow(document.split(','))
    topics = model.get_document_topics(doc_bow)
    if topics:
        return topics[0]
    else:
        return None

df['topic_and_score'] = df['Review_tokenized'].apply(get_topic_and_score)


df['topics'] = df['topic_and_score'].apply(lambda x: x[0][0] if x else None)
df['score'] = df['topic_and_score'].apply(lambda x: x[0][1] if x else None)

df.drop(columns=['topic_and_score'], inplace=True)


In [23]:
df.tail()

,Review ID,Review,Review_tokenized,topics,score
5,6,มาพักครั้งแรก ประทับใจการบริการและการต้อนรับ ท...,พัก,None,None
6,7,ป็นโรงแรมที่คิดมาตลอดว่า ต้องมาพักดูสักคืน โดย...,ป็น,None,None
7,8,เข้าพักทั้งครอบครัวเราพักที่นี่ประจำ มาวันธรรม...,พัก,None,None
8,9,ห้องสวยมาก แต่หาดเฉยๆ เหมาะกับการมาเปลี่ยนที่น...,ห้อง,None,None
9,10,ประทับใจบริการทุกครั้งที่ได้ไปพัก ทุกแผนกเลย ห...,ประทับใจ,None,None
